In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 18 17:27:22 2016

@author: RMFit
"""

### imports ###
import csv

#pandas for data handling and processing
import pandas as pd

#seaborn for visualization
import seaborn as sns

#numpy for mathematical tools
import numpy as np

#matplotlib for plotting
import matplotlib.pylab as plt
import matplotlib
matplotlib.style.use('ggplot')

#sklearn for machine learning
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
#load dataset
df_data = pd.read_csv("dataset/train.csv", dtype={"Age": np.float64}, )
df_test = pd.read_csv("dataset/test.csv", dtype={"Age": np.float64}, )

In [3]:
def processDF(df):
    df['Family'] = df['SibSp'] + df['Parch']
    df.loc[df['Family'] > 0, 'Family'] = 1
    
    #convert Sex to numbers
    df['Female'] = False
    df.loc[df['Sex'] == 'female', 'Female'] = True
    
    #df_data = df_data.drop('PassengerId', axis=1)
    df = df.drop('Name', axis=1)
    df = df.drop('Embarked', axis=1)
    df = df.drop('Cabin', axis=1)
    df = df.drop('SibSp', axis=1)
    df = df.drop('Parch', axis=1)
    df = df.drop('Sex', axis=1)
    df['Ticket'] = pd.to_numeric(df['Ticket'], errors='coerce')
    FareDiv = df['Fare'].mean() + (df['Fare'].mean()/2)
    df['Qclass'] = -1
    df.loc[df['Pclass'] == 3, 'Qclass'] = 1
    df.loc[(df['Pclass'] == 2) & (df['Fare'] > FareDiv), 'Qclass'] = 1
    df['Adult'] = 0
    df.loc[df['Age'] >= 16, 'Adult'] = 1
    df['FareClass'] = 0
    df['FareClass'] = df['Fare'] * df['Qclass']
    
    #normalize data
    exCol = ['PassengerId', 'Survived']
    for i in df.columns:
        if i not in exCol:
            ave = df[i].mean()
            ran = df[i].max() - df[i].min()
            df[i] = (df[i] - ave) / ran
    return df



In [4]:
df_data = processDF(df_data)
df_test = processDF(df_test)

sns.heatmap(df_data.corr())
print(df_data.corr())
#pd.scatter_matrix(df_data)


             PassengerId  Survived    Pclass       Age    Ticket      Fare  \
PassengerId     1.000000 -0.005007 -0.035144  0.036847 -0.076043  0.012658   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.122464  0.257307   
Pclass         -0.035144 -0.338481  1.000000 -0.369226  0.275151 -0.549500   
Age             0.036847 -0.077221 -0.369226  1.000000 -0.148640  0.096067   
Ticket         -0.076043 -0.122464  0.275151 -0.148640  1.000000 -0.139868   
Fare            0.012658  0.257307 -0.549500  0.096067 -0.139868  1.000000   
Family         -0.057462  0.203367 -0.135207 -0.198270  0.085057  0.271832   
Female         -0.042939  0.543351 -0.131900 -0.093254 -0.046484  0.182333   
Qclass         -0.028210 -0.326065  0.912433 -0.316136  0.267088 -0.401674   
Adult           0.029170 -0.006102 -0.229730  0.582875 -0.022111  0.085989   
FareClass      -0.016515 -0.305953  0.690518 -0.195914  0.238675 -0.900900   

               Family    Female    Qclass     Adult  FareClass 

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.


In [5]:
evalColumns = ['Female', 'FareClass', 'Family',]

df_train = pd.DataFrame()
df_final = pd.DataFrame()
for column in evalColumns:
    df_train[column] = df_data[column]
    df_final[column] = df_test[column]

np_train = df_train.values
np_labels = df_data["Survived"].values

train_data, test_data, train_label, test_label = train_test_split(np_train, np_labels, test_size=0.33, random_state=1)

In [6]:
dTree = DecisionTreeClassifier(min_samples_leaf=8)
dTree.fit(train_data, train_label)
dTreePred = dTree.predict(test_data)
print("Accuracy: ", accuracy_score(dTreePred, test_label),)
print("  f1:        ", f1_score(dTreePred, test_label))
print("Recall:   ", recall_score(dTreePred, test_label),)
print("  precision: ", precision_score(dTreePred, test_label))
print(dTree.feature_importances_)

('Accuracy: ', 0.78983050847457625)
('  f1:        ', 0.71028037383177567)
('Recall:   ', 0.81720430107526887)
('  precision: ', 0.62809917355371903)
[ 0.62877426  0.36066761  0.01055814]


In [7]:
rForest = RandomForestClassifier(min_samples_leaf=8)
rForest.fit(train_data, train_label)
rForestPred = rForest.predict(test_data)
print("Accuracy: ", accuracy_score(rForestPred, test_label),)
print("  f1:        ", f1_score(rForestPred, test_label))
print("Recall:   ", recall_score(rForestPred, test_label),)
print("  precision: ", precision_score(rForestPred, test_label))
print(rForest.feature_importances_)

('Accuracy: ', 0.80338983050847457)
('  f1:        ', 0.72115384615384603)
('Recall:   ', 0.86206896551724133)
('  precision: ', 0.6198347107438017)
[ 0.47914667  0.44154671  0.07930662]


In [8]:
df_final = df_final.fillna(value=0)
np_final = df_final.values
output = dTree.predict(df_final).astype(int)
print df_final.columns
ids = df_final['PassengerId'].astype(int)

Index([u'Female', u'FareClass', u'Family'], dtype='object')


KeyError: 'PassengerId'

In [ ]:
predictions_file = open("dTreePred2c.csv", "wb")
open_file_object = csv.DictWriter(predictions_file, fieldnames=['PassengerId','Survived'])
open_file_object.writeheader()
for i in range(len(output)):
    open_file_object.writerows({'PassengerId': ids[i], 'Survived': output[i]})
predictions_file.close()